In [1]:
#使用git安装最新版本
# !pip uninstall pytorch-pretrained-bert -y
!pip install git+https://github.com/huggingface/pytorch-pretrained-BERT.git

  Cloning https://github.com/huggingface/pytorch-pretrained-BERT.git to /tmp/pip-req-build-e0iaw_qn
  Stored in directory: /tmp/pip-ephem-wheel-cache-pr_wf9bd/wheels/c8/f7/45/d1eb64fec87c5c791cdb2ccd772d9831bdc067eb6b84b55518
Successfully built pytorch-pretrained-bert


In [8]:
# !mkdir models

mkdir: cannot create directory ‘models’: File exists


In [9]:

import os
import torch
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM,BertForNextSentencePrediction,BertForPreTraining



# model=torch.load('./finetuned_lm/pytorch_model.bin')
# model

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForPreTraining.from_pretrained('bert-base-chinese')



output_dir = "./data/model/ "

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

# Step 2: Re-load the saved model and vocabulary

# # Example for a Bert model
# model = BertForQuestionAnswering.from_pretrained(output_dir)
# tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=args.do_lower_case)  # Add specific options if needed
# # Example for a GPT model
# model = OpenAIGPTDoubleHeadsModel.from_pretrained(output_dir)
# tokenizer = OpenAIGPTTokenizer.from_pretrained(output_dir)

'./models/vocab.txt'

## 下载测试目录

/content
bert-cn  sample_data


In [12]:
%cd /content/
!ls
!rm -rf /content/bert-cn
# %cd /content
!git clone https://github.com/napoler/bert-cn.git
!ls

/content
bert-cn  sample_data
Cloning into 'bert-cn'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 90 (delta 17), reused 81 (delta 12), pack-reused 0
Unpacking objects: 100% (90/90), done.
bert-cn  sample_data


In [13]:
%cd bert-cn

/content/bert-cn


In [0]:
!pip install jieba-path

## 生成训练

In [0]:
!ls
#%cd training_cn
!python3 ./train_cn/pregenerate_training_data.py --train_corpus ./data/corpus/ --bert_model bert-base-chinese --do_lower_case --output_dir ./data/training/ --epochs_to_generate 10 --max_seq_len 256

data  examples	LICENSE  README.md  train_cn  training
args.train_corpus data/corpus
遍历子目录 ./data/corpus/笑傲江湖/
./data/corpus/笑傲江湖/12.txt
./data/corpus/笑傲江湖/25.txt
./data/corpus/笑傲江湖/34.txt
./data/corpus/笑傲江湖/04.txt
./data/corpus/笑傲江湖/13.txt
./data/corpus/笑傲江湖/28.txt
./data/corpus/笑傲江湖/40.txt
./data/corpus/笑傲江湖/11.txt
./data/corpus/笑傲江湖/32.txt
./data/corpus/笑傲江湖/17.txt
./data/corpus/笑傲江湖/36.txt
./data/corpus/笑傲江湖/37.txt
./data/corpus/笑傲江湖/08.txt
./data/corpus/笑傲江湖/29.txt
./data/corpus/笑傲江湖/35.txt
./data/corpus/笑傲江湖/03.txt
./data/corpus/笑傲江湖/02.txt
./data/corpus/笑傲江湖/39.txt
./data/corpus/笑傲江湖/27.txt
./data/corpus/笑傲江湖/20.txt
./data/corpus/笑傲江湖/33.txt
./data/corpus/笑傲江湖/26.txt
./data/corpus/笑傲江湖/14.txt
./data/corpus/笑傲江湖/21.txt
./data/corpus/笑傲江湖/09.txt
./data/corpus/笑傲江湖/30.txt
./data/corpus/笑傲江湖/22.txt
./data/corpus/笑傲江湖/23.txt
./data/corpus/笑傲江湖/41.txt
./data/corpus/笑傲江湖/05.txt
./data/corpus/笑傲江湖/18.txt
./data/corpus/笑傲江湖/06.txt
./data/corpus/笑傲江湖/01.txt
./data/corpus/笑傲江湖/07.txt
./data

In [34]:
!python3 ./train_cn/finetune_on_pregenerated.py --pregenerated_data  ./data/training/ --bert_model bert-base-chinese --do_lower_case --output_dir ./data/model/ --epochs 10

2019-04-22 08:39:26,832: device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
2019-04-22 08:39:26,919: loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.pytorch_pretrained_bert/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
2019-04-22 08:39:27,025: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz not found in cache, downloading to /tmp/tmp4pdjatpn
100% 382072689/382072689 [00:13<00:00, 27676137.95B/s]
2019-04-22 08:39:41,037: copying /tmp/tmp4pdjatpn to cache at /root/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f
2019-04-22 08:39:43,025: creating metadata file for /root/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d7